In [ ]:
Park

In [ ]:
import pandas as pd

parks = pd.read_csv('../Project 2/parks_in_victoria.csv')
print(parks.head())

In [ ]:
print(len(parks))
print(parks.columns)
print(parks.head)

In [ ]:
parks['geometry'] = parks.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

In [ ]:
domain = pd.read_csv('../project 2/updated_domain_with_lat_long.csv')

In [ ]:
from shapely.geometry import Point

domain['geometry'] = domain.apply(lambda x: Point((x['Longitude'], x['Latitude'])), axis=1)
domain_gdf = gpd.GeoDataFrame(domain, geometry='geometry', crs="EPSG:4326")
domain_gdf_valid = domain_gdf[~domain_gdf.is_empty]

parks['geometry'] = parks.apply(lambda x: Point((x['Longitude'], x['Latitude'])), axis=1)
parks_gdf = gpd.GeoDataFrame(parks, geometry='geometry', crs="EPSG:4326")

domain_gdf_valid_3857 = domain_gdf_valid.to_crs(epsg=3857)
parks_gdf_3857 = parks_gdf.to_crs(epsg=3857)

domain_gdf_with_park = gpd.sjoin_nearest(domain_gdf_valid_3857, parks_gdf_3857, how='left', max_distance=1500)


domain_gdf_with_park['has_park_nearby'] = domain_gdf_with_park['Name'].notnull()
true_nearby_parks = domain_gdf_with_park[domain_gdf_with_park['has_park_nearby'] == True]

print(true_nearby_parks.columns)
print(true_nearby_parks.head)

In [ ]:
true_nearby_parks.head()

In [ ]:
true_nearby_parks_cleaned = true_nearby_parks.drop_duplicates()

In [ ]:
import googlemaps

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='Your Key')

def calculate_driving_distance(row):

    apartment_coords = f"{row['Latitude_left']},{row['Longitude_left']}"
    parks_coords = f"{row['Latitude_right']},{row['Longitude_right']}"
    
    try:
        result = gmaps.distance_matrix(apartment_coords, parks_coords, mode='driving')
        distance_meters = result['rows'][0]['elements'][0]['distance']['value'] 
        distance_km = distance_meters / 1000  
        return distance_km
    except Exception as e:
        print(f"Error calculating distance for {row['Address_left']}: {e}")
        return None

# Apply the function to each row to calculate the driving distance from the apartment to the mall
true_nearby_parks_cleaned['distance_to_park'] = true_nearby_parks_cleaned.apply(calculate_driving_distance, axis=1)

# Display updated DataFrame with address and driving distance to nearest mall
print(true_nearby_parks_cleaned[['Address_left', 'distance_to_park']].head())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(true_nearby_parks_cleaned['distance_to_park'], bins=30, edgecolor='black')

plt.title('Distribution of Distance to park', fontsize=15)
plt.xlabel('Distance to malls (km)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

In [ ]:
correlation = true_nearby_parks_cleaned[['Weekly Price', 'distance_to_park']].corr()

print("Correlation between Weekly Price and Distance to Train:")
print(correlation)

In [ ]:
house_data = true_nearby_parks_cleaned[true_nearby_parks_cleaned['Type'] != 'Apartment']

correlation_house = house_data[['Weekly Price', 'distance_to_park']].corr()

print("Correlation between Weekly Price and Distance to Train:")
print(correlation_house)